In [8]:
import numpy as np

In [3]:
class FluidVars:
    """A simple container that holds the integer indicies we will use to
    refer to the different fluid components"""
    def __init__(self):
        self.nvar = 3
    
        # conserved variables
        self.urho = 0
        self.umx = 1
        self.uener = 2
    
        # primitive variables
        self.qrho = 0
        self.qu = 1
        self.qp = 2

In [7]:
class Euler:
    def __init__(self, nx, C, *,
                 gamma=1.4, init_cond=None):

        self.grid = FVGrid(nx, ng=4)
        self.v = FluidVars()
        
        self.C = C

        # storage for the current solution
        self.U = self.grid.scratch_array(nc=self.v.nvar)

        # initialize
        init_cond(self.grid, self.U)

    def estimate_dt(self):
        pass

    def cons_to_prim(self, U):
        q = self.grid.scratch_array(nc=self.v.nvar)
        
        q[:, self.v.qrho] = U[:, self.v.urho]
        q[:, self.v.qu] = U[:, self.v.umx] / U[:. self.v.urho]
        
        rhoe = U[:, self.v.uener] - 0.5 * q[:, self.v.qu]**2
        q[:, self.v.qp] = rhoe * (self.gamma - 1.0)
        
        return q

    def interface_states(self):

        # compute flattening

        # construct parabola

        # loop over zones

        # build eigensystem

        # trace states


    def advance(self):

        # convert to primitive variables

        # construct interface states

        # solve Riemann problem

        # construct fluxes

        # conservative update




1

In [5]:
rho

NameError: name 'rho' is not defined

In [6]:
Prim.rho

<Prim.rho: 1>